In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.optimize import curve_fit
from datetime import datetime, timedelta
data=pd.read_csv("C:/Users/20195478/Documents/Hackathon/DataSetNetherlands.csv")
#data=data.drop('Last Update', axis=1)
#data=data.drop("SNo",axis=1)
data=data.rename(columns={"Date": "date", " Cityname": "city", " Provincename": "state"," Number":"confirm"})

list(data.columns)
data.head(5)

,date,city,Citycode,state,confirm,weekday,month,day
0,2020-03-31,Appingedam,3,Groningen,1.0,Tuesday,3.0,31.0
1,2020-03-31,Waalwijk,867,North Brabant,16.0,Tuesday,3.0,31.0
2,2020-03-31,Vught,865,North Brabant,23.0,Tuesday,3.0,31.0
3,2020-03-31,Veldhoven,861,North Brabant,7.0,Tuesday,3.0,31.0
4,2020-03-31,Valkenswaard,858,North Brabant,6.0,Tuesday,3.0,31.0


In [46]:
def plot_predict(city, stat, future_days):
    def avg_err(pcov):
        return np.round(np.sqrt(np.diag(pcov)).mean(), 2)
    # function to be minimized
    def f_sigmoid(x, a, b, c):
        # a = sigmoid midpoint
        # b = curve steepness (logistic growth)
        # c = max value
        return (c / (1 + np.exp(-b*(x-a))))
  
    inception = 0
    # hardcoding day 0 for several countries based on observations
   
   # if city=="Veldhoven": inception = 20
    city_data = data[data["city"]==city].copy()
    city_graph = city_data.groupby("date")[['confirm']].sum().reset_index()[inception:]
    y = city_graph[stat]
    x = np.arange(len(y))
    
    # fitting the data on the logistic function
    popt_sig, pcov_sig = curve_fit(f_sigmoid, x, y, method='dogbox', bounds=([12., 0.001, y.mean()],[60., 2.5, 10*y.max()]))
    print(popt_sig)
    peakday = datetime.strftime(datetime.strptime(city_graph["date"][inception], "%m/%d/%Y")+timedelta(days=int(popt_sig[0])), "%m/%d/%Y")
    plt.figure(figsize=(16,8))
    
    x_m = np.arange(len(y)+future_days)
    y_m = f_sigmoid(x_m, *popt_sig)
    print("Predictions:")
    for i in range(1,9):
        pday = datetime.strftime(datetime.strptime(city_graph["date"][inception], "%m/%d/%Y")+timedelta(days=len(y)+i-1), "%m/%d/%Y")
        print("%s: %d" % (pday, y_m[len(y)+i-1]))
    #print(country_graph)
    
    # creating the matplotlib visualization
    plt.plot(x_m, y_m, c='k', marker="x", label="sigmoid | error: "+str(avg_err(pcov_sig))) 
    plt.text(x_m[-1]+.5, y_m[-1], str(int(y_m[-1])), size = 10)
    
    plt.plot(x, y, c='r', marker="o", label = stat)
    plt.xlabel("Days")
    plt.ylabel("Total Infected")
    plt.legend(prop={'size': 15})
    plt.title(country+"'s Data", size=15)
    plt.axvline(x[-1])
    plt.text(x[-1]-.5, y_m[-1], str(city_graph["date"][len(y)+inception-1]), size = 10)
    plt.axvline(int(popt_sig[0]))
    plt.text(int(popt_sig[0]), 1, "peak: day " + str(int(popt_sig[0])) + " (" + peakday + ")", size = 10)
    plt.show()
# See the results for different countries
#plot_predict("Mainland China", "confirm", 10)
#plot_predict("South Korea", "confirm", 10)
plot_predict("Veldhoven", "confirm", 10)

ValueError: `ydata` must not be empty!